In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

prompts = [
    "What is C. elegans?",
    "Say hi in JSON format",
    "Define AI in French"
]

model_name = "LiquidAI/LFM2-350M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda")
model.to(device)

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.3,
        repetition_penalty=1.05
    )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")